In [ ]:
import asyncio
import json
import pyaudio
import websockets
import ssl
import nest_asyncio
from openai import OpenAI
from collections import deque
import requests
import sounddevice as sd
import soundfile as sf
from io import BytesIO
import time
import wave
import sqlite3
import faiss
import numpy as np
import torch
import warnings
from sentence_transformers import SentenceTransformer, CrossEncoder
import pickle
from typing import List, Dict, Tuple
from tqdm.notebook import tqdm

# 禁用特定类型的警告
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 允许嵌套事件循环
nest_asyncio.apply()

class MentalHealthRAG:
    def __init__(
        self,
        db_path: str = "mental_health_resources.db",
        embedding_model_name: str = "shibing624/text2vec-base-chinese",
        reranker_model_name: str = "BAAI/bge-reranker-base",
        index_path: str = "faiss_index.bin",
        resources_path: str = "resources.pkl",
        debug: bool = True
    ):
        self.db_path = db_path
        print(f"\n加载向量化模型: {embedding_model_name}")
        self.embedding_model = SentenceTransformer(embedding_model_name)
        print(f"加载重排序模型: {reranker_model_name}")
        self.reranker = CrossEncoder(reranker_model_name)
        self.index_path = index_path
        self.resources_path = resources_path
        self.index = None
        self.resources = []
        self.debug = debug

    def load_resources(self) -> List[Dict]:
        """从数据库加载资源"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM resources")
        resources = cursor.fetchall()
        conn.close()

        return [
            {
                'id': r[0],
                'title': r[1],
                'summary': r[2],
                'link': r[3],
                'text': f"{r[1]} {r[2]}"
            }
            for r in resources
        ]

    def build_index(self):
        """构建FAISS索引"""
        print("加载资源...")
        self.resources = self.load_resources()
        
        print("生成文本向量...")
        texts = [r['text'] for r in self.resources]
        embeddings = self.embedding_model.encode(
            texts,
            show_progress_bar=True,
            convert_to_numpy=True,
            # 使用 notebook 专用的进度条
            progress_bar_class=tqdm
        )

        print("构建FAISS索引...")
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings.astype('float32'))

        print("保存索引和资源数据...")
        faiss.write_index(self.index, self.index_path)
        with open(self.resources_path, 'wb') as f:
            pickle.dump(self.resources, f)

        print("索引构建完成！")

    def load_index(self):
        """加载已存在的索引和资源"""
        self.index = faiss.read_index(self.index_path)
        with open(self.resources_path, 'rb') as f:
            self.resources = pickle.load(f)

    def search(
        self,
        query: str,
        k: int = 3,
        rerank: bool = True,
        threshold: float = 0.5
    ) -> List[Dict]:
        """搜索相关资源"""
        if self.debug:
            print("\nRAG检索过程:")
            print(f"查询文本: {query}")
        
        # 计算查询向量
        query_vector = self.embedding_model.encode([query])
        
        # FAISS检索
        distances, indices = self.index.search(
            query_vector.astype('float32'), 
            k
        )
        
        if self.debug:
            print("\n初始检索结果:")
        
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            if idx != -1:
                result = self.resources[idx].copy()
                similarity = 1 / (1 + distance)  # 将L2距离转换为相似度
                result['initial_score'] = similarity
                results.append(result)
                if self.debug:
                    print(f"- {result['title']}")
                    print(f"  初始相似度: {similarity:.4f}")
                    print(f"  L2距离: {distance:.4f}")
        
        if rerank and results:
            if self.debug:
                print("\n重排序过程:")
            
            # 准备重排序的文本对
            pairs = [(query, r['text']) for r in results]
            
            # 计算相关性分数
            scores = self.reranker.predict(pairs)
            
            # 重排序并过滤
            reranked = []
            for score, result in zip(scores, results):
                if score >= threshold:
                    result['rerank_score'] = float(score)
                    reranked.append(result)
                    if self.debug:
                        print(f"- {result['title']}")
                        print(f"  重排序分数: {score:.4f}")
                        print(f"  是否通过阈值({threshold}): {'是' if score >= threshold else '否'}")
            
            # 按分数降序排序
            reranked.sort(key=lambda x: x['rerank_score'], reverse=True)
            
            if self.debug:
                print(f"\n最终返回 {len(reranked)} 个结果")
            return reranked
            
        return results

    def get_suggestion(self, conversation_text: str) -> Tuple[bool, List[Dict]]:
        """根据对话内容判断是否需要推荐资源"""
        if self.debug:
            print("\n资源推荐流程开始:")
            
        triggers = {
            '抑郁': 0.6,
            '焦虑': 0.6,
            '睡眠': 0.6,
            '压力': 0.5,
            '药物': 0.7,
            '家属': 0.5,
            '量表': 0.8,
            '检查': 0.6,
            '治疗': 0.7,
            '症状': 0.6
        }
        
        # 检查触发词
        matched_triggers = [
            (word, threshold) 
            for word, threshold in triggers.items() 
            if word in conversation_text
        ]
        
        if self.debug:
            if matched_triggers:
                print("检测到以下触发词:")
                for word, threshold in matched_triggers:
                    print(f"- {word} (阈值: {threshold})")
            else:
                print("未检测到触发词")
        
        if not matched_triggers:
            return False, []
            
        # 使用最高阈值
        max_threshold = max(threshold for _, threshold in matched_triggers)
        if self.debug:
            print(f"使用最高阈值: {max_threshold}")
            
        results = self.search(
            conversation_text,
            k=3,
            rerank=True,
            threshold=max_threshold
        )
        
        return bool(results), results

class MicrophoneClient:
    def __init__(
        self, 
        asr_url="ws://127.0.0.1:10095",  # New parameter for ASR URL
        tts_url="http://127.0.0.1:9880",
        deepseek_api_key=None, 
        db_path="mental_health_resources.db",
        debug=True
    ):
        # Parse ASR URL
        self.asr_url = asr_url
        if asr_url.startswith('wss://'):
            self.use_ssl = True
            self.host = asr_url[6:].split(':')[0]
        elif asr_url.startswith('ws://'):
            self.use_ssl = False
            self.host = asr_url[5:].split(':')[0]
        try:
            self.port = int(asr_url.split(':')[-1])
        except (IndexError, ValueError):
            self.port = 443 if self.use_ssl else 80

        self.chunk_size = [5, 10, 5]
        self.chunk_interval = 10
        self.websocket = None
        self.last_text = ""
        self.tts_url = tts_url
        self.recording = True
        self.stream = None
        self.audio = None
        self.debug = debug
        
        # 初始化DeepSeek客户端
        self.deepseek_client = OpenAI(
            api_key=deepseek_api_key, 
            base_url="https://api.deepseek.com"
        ) if deepseek_api_key else None
        
        # 存储对话历史
        self.conversation_history = deque(maxlen=10)
        
        # 初始化RAG系统
        self.rag = MentalHealthRAG(db_path=db_path, debug=debug)
        try:
            print("加载RAG索引...")
            self.rag.load_index()
        except (FileNotFoundError, Exception) as e:
            print("RAG索引不存在，正在构建新索引...")
            self.rag.build_index()
        
    def play_audio(self, wav_data):
        """播放音频数据"""
        try:
            wav_io = BytesIO(wav_data)
            wav_io.seek(0)
            data, samplerate = sf.read(wav_io)
            sd.play(data, samplerate, blocking=True)
            time.sleep(0.3)
        except Exception as e:
            print(f"音频播放错误: {e}")

    async def text_to_speech(self, text):
        """调用TTS API将文本转换为语音"""
        try:
            self.recording = False
            if self.stream:
                self.stream.stop_stream()
                await asyncio.sleep(1.0)
            
            payload = {
                "text": text,
                "text_language": "zh",
            }
            
            print("\n正在生成语音...")
            response = requests.post(self.tts_url, json=payload)
            
            if response.status_code == 200:
                print("播放中...")
                await asyncio.get_event_loop().run_in_executor(
                    None, 
                    self.play_audio, 
                    response.content
                )
                await asyncio.sleep(1.5)
            else:
                print(f"TTS API调用失败: {response.status_code}")
            
            print("准备继续录音...")
            await asyncio.sleep(1.5)
            if self.stream:
                self.stream.start_stream()
            self.recording = True
            print("录音已恢复")
                
        except Exception as e:
            print(f"TTS转换错误: {e}")
            if self.stream:
                self.stream.start_stream()
            self.recording = True

    async def format_resource_recommendations(self, resources):
        """格式化资源推荐为语音输出友好的格式"""
        if not resources:
            return ""
            
        recommendations = "\n\n根据您的描述，我为您推荐以下参考资源："
        for r in resources:
            recommendations += f"\n\n{r['title']}"
            if 'rerank_score' in r:
                recommendations += f"（相关度：{r['rerank_score']:.2%}）"
            recommendations += f"\n{r['summary']}"
        return recommendations

    async def get_ai_response(self, text):
        """获取AI回复并集成资源推荐"""
        if not self.deepseek_client:
            return "未配置DeepSeek API密钥"
            
        try:
            # 获取基础AI回复
            messages = [
                {"role": "system", "content": (
                    "你是回龙观医院心理科王医生，请针对患者或患者家属的提问给出合适的回复。"
                    "回复要言简意赅，控制在100字以内。"
                )}
            ]
            
            for history in self.conversation_history:
                messages.append({"role": "user", "content": history["user"]})
                messages.append({"role": "assistant", "content": history["assistant"]})
            
            messages.append({"role": "user", "content": text})
            
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.deepseek_client.chat.completions.create(
                    model="deepseek-chat",
                    messages=messages,
                    max_tokens=400,
                    stream=False
                )
            )
            ai_reply = response.choices[0].message.content
            
            # 检查是否需要推荐资源
            should_recommend, resources = self.rag.get_suggestion(text + " " + ai_reply)
            
            if should_recommend:
                # 添加资源推荐
                recommendations = await self.format_resource_recommendations(resources)
                ai_reply += recommendations
            
            # 存储当前对话
            self.conversation_history.append({
                "user": text,
                "assistant": ai_reply
            })
            
            return ai_reply
        except Exception as e:
            return f"获取AI回复失败: {str(e)}"
        
    async def start_streaming(self):
        MAX_RETRIES = 3
        RETRY_DELAY = 2
        retry_count = 0

        while retry_count < MAX_RETRIES:
            try:
                uri = f"{'wss' if self.use_ssl else 'ws'}://{self.host}:{self.port}"
                ssl_context = None
                if self.use_ssl:
                    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
                    ssl_context.check_hostname = False
                    ssl_context.verify_mode = ssl.CERT_NONE
                
                print(f"正在连接服务器 {uri}...")
                async with websockets.connect(
                    uri, 
                    subprotocols=["binary"], 
                    ping_interval=None, 
                    ssl=ssl_context
                ) as self.websocket:
                    print("连接成功！")
                    await asyncio.gather(
                        self._record_microphone(),
                        self._receive_messages()
                    )
                break
            except (websockets.exceptions.WebSocketException, TimeoutError, ConnectionRefusedError) as e:
                retry_count += 1
                if retry_count < MAX_RETRIES:
                    print(f"连接失败: {str(e)}")
                    print(f"将在 {RETRY_DELAY} 秒后进行第 {retry_count + 1} 次重试...")
                    await asyncio.sleep(RETRY_DELAY)
                else:
                    print(f"连接失败，已达到最大重试次数 ({MAX_RETRIES})。")
                    print("请检查:")
                    print("1. 语音识别服务是否已启动")
                    print(f"2. 服务器地址 ({self.host}) 和端口 ({self.port}) 是否正确")
                    print("3. 网络连接是否正常")
                    print(f"详细错误: {str(e)}")
                    raise
    
    async def _record_microphone(self):
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 16000
        chunk_size = 60 * self.chunk_size[1] / self.chunk_interval
        CHUNK = int(RATE / 1000 * chunk_size)
        
        self.audio = pyaudio.PyAudio()
        self.stream = self.audio.open(
            format=FORMAT,
            channels=CHANNELS,
            rate=RATE,
            input=True,
            frames_per_buffer=CHUNK
        )
        
        config_message = json.dumps({
            "mode": "2pass",
            "chunk_size": self.chunk_size,
            "chunk_interval": self.chunk_interval,
            "wav_name": "microphone",
            "is_speaking": True,
            "hotwords": "",
            "itn": True
        })
        await self.websocket.send(config_message)
        
        try:
            while True:
                if self.recording:
                    try:
                        data = self.stream.read(CHUNK)
                        await self.websocket.send(data)
                    except Exception as e:
                        if self.recording:
                            print(f"录音错误: {e}")
                await asyncio.sleep(0.005)
        except Exception as e:
            print(f"录音循环错误: {e}")
        finally:
            if self.stream:
                self.stream.stop_stream()
                self.stream.close()
            if self.audio:
                self.audio.terminate()
    
    async def _receive_messages(self):
        try:
            while True:
                message = await self.websocket.recv()
                try:
                    msg_data = json.loads(message)
                    if "text" in msg_data:
                        new_text = msg_data["text"]
                        mode = msg_data.get("mode", "")
                        
                        if len(new_text.strip()) == 0:
                            continue
                            
                        if mode == "2pass-online":
                            if new_text != self.last_text:
                                print(f"\r实时识别: {new_text}", end="")
                                self.last_text = new_text
                        elif mode == "2pass-offline":
                            print(f"\nVAD结果: {new_text}")
                            # 获取AI回复（包含资源推荐）
                            ai_response = await self.get_ai_response(new_text)
                            print(f"AI回复: {ai_response}\n")
                            # 转换为语音并播放
                            await self.text_to_speech(ai_response)
                            print(f"[对话历史: {len(self.conversation_history)}轮]\n")
                            self.last_text = ""
                            
                except json.JSONDecodeError:
                    continue
                except Exception as e:
                    print(f"处理消息错误: {e}")
        except Exception as e:
            print(f"接收消息错误: {e}")

def start_mic_client(
    deepseek_api_key=None, 
    tts_url="http://127.0.0.1:9880",
    asr_url="wss://127.0.0.1:10095",  # New parameter
    db_path="mental_health_resources.db",
    debug=True
):
    """启动麦克风客户端的便捷函数"""
    client = MicrophoneClient(
        deepseek_api_key=deepseek_api_key, 
        tts_url=tts_url,
        asr_url=asr_url,  # Pass ASR URL to client
        db_path=db_path,
        debug=debug
    )
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(client.start_streaming())
    except KeyboardInterrupt:
        print("\n录音已停止")
    except Exception as e:
        print(f"发生错误: {e}")
        print("\n如需重新启动，请再次运行程序。")
    finally:
        if hasattr(client, 'stream') and client.stream:
            client.stream.stop_stream()
            client.stream.close()
        if hasattr(client, 'audio') and client.audio:
            client.audio.terminate()

# 使用示例
if __name__ == "__main__":
    # 配置参数
    DEEPSEEK_API_KEY = "sk-a753a785ce4f4d418de4caf17e82b629"
    TTS_URL = "http://copilot.mynatapp.cc"
    ASR_URL = "wss://693e14e788532c17.natapp.cc:54321"  # Updated ASR URL configuration
    DB_PATH = "mental_health_resources.db"
    
    print("开始连接语音服务...")
    print(f"语音识别服务器: {ASR_URL}")
    print(f"TTS服务器: {TTS_URL}")
    
    start_mic_client(
        deepseek_api_key=DEEPSEEK_API_KEY,
        tts_url=TTS_URL,
        asr_url=ASR_URL,  # Pass ASR URL
        db_path=DB_PATH,
        debug=True
    )

In [1]:
import asyncio
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import warnings
import pyaudio
import websockets
import ssl
from openai import OpenAI
from collections import deque
import sounddevice as sd
import soundfile as sf
from io import BytesIO
import sqlite3
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import pickle
import requests
from contextlib import asynccontextmanager

# 禁用警告
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

@dataclass
class AudioConfig:
    """音频配置类"""
    format: int = pyaudio.paInt16
    channels: int = 1
    rate: int = 16000
    chunk_size: List[int] = None
    chunk_interval: int = 10

    def __post_init__(self):
        self.chunk_size = self.chunk_size or [5, 10, 5]
        self.buffer_size = int(self.rate / 1000 * (60 * self.chunk_size[1] / self.chunk_interval))

class RAGSystem:
    """检索增强生成系统"""
    def __init__(
        self,
        db_path: str,
        embedding_model: str = "shibing624/text2vec-base-chinese",
        reranker_model: str = "BAAI/bge-reranker-base",
        index_path: str = "faiss_index.bin",
        resources_path: str = "resources.pkl",
        debug: bool = True
    ):
        self.db_path = db_path
        self.embedding_model = SentenceTransformer(embedding_model)
        self.reranker = CrossEncoder(reranker_model)
        self.index_path = index_path
        self.resources_path = resources_path
        self.debug = debug
        self.index = None
        self.resources = []
        self._initialize_system()

    def _initialize_system(self):
        """初始化系统"""
        try:
            print("加载RAG索引...")
            self._load_index()
        except FileNotFoundError:
            print("RAG索引不存在，正在构建新索引...")
            self._build_index()
        print("索引加载完成！")

    def _load_resources(self) -> List[Dict]:
        """从数据库加载资源"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM resources")
            return [{
                'id': r[0],
                'title': r[1],
                'summary': r[2],
                'link': r[3],
                'text': f"{r[1]} {r[2]}"
            } for r in cursor.fetchall()]

    def _build_index(self):
        """构建FAISS索引"""
        print("加载资源...")
        self.resources = self._load_resources()
        
        print("生成文本向量...")
        embeddings = self.embedding_model.encode(
            [r['text'] for r in self.resources],
            convert_to_numpy=True,
            show_progress_bar=True
        )
        
        print("构建FAISS索引...")
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings.astype('float32'))
        
        print("保存索引和资源数据...")
        faiss.write_index(self.index, self.index_path)
        with open(self.resources_path, 'wb') as f:
            pickle.dump(self.resources, f)
            
        print("索引构建完成！")

    def _load_index(self):
        """加载索引"""
        self.index = faiss.read_index(self.index_path)
        with open(self.resources_path, 'rb') as f:
            self.resources = pickle.load(f)

    def search(
        self,
        query: str,
        k: int = 3,
        rerank: bool = True,
        threshold: float = 0.5
    ) -> List[Dict]:
        """搜索相关资源"""
        if self.debug:
            print("\nRAG检索过程:")
            print(f"查询文本: {query}")
            
        query_vector = self.embedding_model.encode([query])
        distances, indices = self.index.search(query_vector.astype('float32'), k)
        
        if self.debug:
            print("\n初始检索结果:")
            
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            if idx != -1:
                result = self.resources[idx].copy()
                similarity = 1 / (1 + distance)
                result['initial_score'] = similarity
                results.append(result)
                
                if self.debug:
                    print(f"- {result['title']}")
                    print(f"  初始相似度: {similarity:.4f}")
                    print(f"  L2距离: {distance:.4f}")
        
        if rerank and results:
            if self.debug:
                print("\n重排序过程:")
                
            # 准备重排序的文本对
            pairs = [(query, r['text']) for r in results]
            
            # 计算相关性分数
            scores = self.reranker.predict(pairs)
            
            # 重排序并过滤
            reranked = []
            for score, result in zip(scores, results):
                if score >= threshold:
                    result['rerank_score'] = float(score)
                    reranked.append(result)
                    if self.debug:
                        print(f"- {result['title']}")
                        print(f"  重排序分数: {score:.4f}")
                        print(f"  是否通过阈值({threshold}): {'是' if score >= threshold else '否'}")
            
            reranked.sort(key=lambda x: x['rerank_score'], reverse=True)
            
            if self.debug:
                print(f"\n最终返回 {len(reranked)} 个结果")
            return reranked
            
        return results

    def get_suggestion(self, text: str) -> Tuple[bool, List[Dict]]:
        """根据文本获取推荐"""
        if self.debug:
            print("\n资源推荐流程开始:")
            
        triggers = {
            '抑郁': 0.6, '焦虑': 0.6, '睡眠': 0.6,
            '压力': 0.5, '药物': 0.7, '家属': 0.5,
            '量表': 0.8, '检查': 0.6, '治疗': 0.7, '症状': 0.6
        }
        
        matched = [(word, thresh) for word, thresh in triggers.items() if word in text]
        if self.debug:
            if matched:
                print("检测到以下触发词:")
                for word, threshold in matched:
                    print(f"- {word} (阈值: {threshold})")
            else:
                print("未检测到触发词")
                
        if not matched:
            return False, []
            
        max_threshold = max(thresh for _, thresh in matched)
        if self.debug:
            print(f"使用最高阈值: {max_threshold}")
            
        results = self.search(text, threshold=max_threshold)
        return bool(results), results

class VoiceAssistant:
    """语音助手类"""
    def __init__(
        self,
        rag_system: RAGSystem,
        ai_client: OpenAI,
        asr_url: str,
        tts_url: str,
        audio_config: AudioConfig = None
    ):
        self.rag = rag_system
        self.ai_client = ai_client
        self.asr_url = asr_url
        self.tts_url = tts_url
        self.audio_config = audio_config or AudioConfig()
        self.conversation_history = deque(maxlen=10)
        self.recording = True
        self.last_text = ""
        self._setup_audio()

    def _setup_audio(self):
        """设置音频设备"""
        self.audio = pyaudio.PyAudio()
        self.stream = self.audio.open(
            format=self.audio_config.format,
            channels=self.audio_config.channels,
            rate=self.audio_config.rate,
            input=True,
            frames_per_buffer=self.audio_config.buffer_size
        )

    async def _play_audio(self, wav_data: bytes):
        """播放音频"""
        wav_io = BytesIO(wav_data)
        data, samplerate = sf.read(wav_io)
        sd.play(data, samplerate, blocking=True)
        await asyncio.sleep(0.3)

    async def text_to_speech(self, text: str):
        """文本转语音"""
        self.recording = False
        if self.stream:
            self.stream.stop_stream()
        
        try:
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: requests.post(
                    self.tts_url,
                    json={"text": text, "text_language": "zh"}
                )
            )
            
            if response.status_code == 200:
                await self._play_audio(response.content)
        finally:
            await asyncio.sleep(1.5)
            if self.stream:
                self.stream.start_stream()
            self.recording = True

    async def get_ai_response(self, text: str) -> str:
        """获取AI回复"""
        messages = [
            {"role": "system", "content": (
                "你是回龙观医院心理科王医生，请针对患者或患者家属的提问给出合适的回复。"
                "回复要言简意赅，控制在100字以内。"
            )}
        ]
        
        # 添加历史对话
        messages.extend([
            {"role": "user", "content": h["user"]}
            for h in self.conversation_history
        ])
        messages.append({"role": "user", "content": text})
        
        try:
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.ai_client.chat.completions.create(
                    model="deepseek-chat",
                    messages=messages,
                    max_tokens=400
                )
            )
            
            ai_reply = response.choices[0].message.content
            
            # 检查是否需要推荐资源
            should_recommend, resources = self.rag.get_suggestion(text + " " + ai_reply)
            if should_recommend:
                ai_reply += self._format_recommendations(resources)
            
            # 保存对话历史
            self.conversation_history.append({"user": text, "assistant": ai_reply})
            return ai_reply
            
        except Exception as e:
            return f"获取AI回复失败: {str(e)}"

    def _format_recommendations(self, resources: List[Dict]) -> str:
        """格式化推荐资源"""
        if not resources:
            return ""
            
        recommendations = "\n\n根据您的描述，我为您推荐以下参考资源："
        for r in resources:
            recommendations += (
                f"\n\n{r['title']}"
                f"（相关度：{r.get('rerank_score', 0):.2%}）"
                f"\n{r['summary']}"
            )
        return recommendations

    @asynccontextmanager
    async def websocket_connection(self):
        """WebSocket连接上下文管理器"""
        ssl_context = None
        if self.asr_url.startswith('wss://'):
            ssl_context = ssl.create_default_context()
            ssl_context.check_hostname = False
            ssl_context.verify_mode = ssl.CERT_NONE
        
        async with websockets.connect(
            self.asr_url,
            subprotocols=["binary"],
            ping_interval=None,
            ssl=ssl_context
        ) as websocket:
            yield websocket

    async def start(self):
        """启动语音助手"""
        async with self.websocket_connection() as websocket:
            # 发送配置
            await websocket.send(json.dumps({
                "mode": "2pass",
                "chunk_size": self.audio_config.chunk_size,
                "chunk_interval": self.audio_config.chunk_interval,
                "wav_name": "microphone",
                "is_speaking": True,
                "hotwords": "",
                "itn": True
            }))
            
            # 启动录音和消息接收
            await asyncio.gather(
                self._record_audio(websocket),
                self._handle_messages(websocket)
            )

    async def _record_audio(self, websocket):
        """录制音频"""
        try:
            while True:
                if self.recording:
                    data = self.stream.read(self.audio_config.buffer_size)
                    await websocket.send(data)
                await asyncio.sleep(0.005)
        finally:
            if self.stream:
                self.stream.stop_stream()
                self.stream.close()
            if self.audio:
                self.audio.terminate()

    async def _handle_messages(self, websocket):
        """处理WebSocket消息"""
        async for message in websocket:
            try:
                msg_data = json.loads(message)
                if "text" not in msg_data:
                    continue
                    
                text = msg_data["text"].strip()
                if not text:
                    continue
                    
                mode = msg_data.get("mode", "")
                if mode == "2pass-online":
                    if text != self.last_text:
                        print(f"\r实时识别: {text}", end="")
                        self.last_text = text
                elif mode == "2pass-offline":
                    print(f"\nVAD结果: {text}")
                    ai_response = await self.get_ai_response(text)
                    print(f"AI回复: {ai_response}\n")
                    await self.text_to_speech(ai_response)
                    print(f"[对话历史: {len(self.conversation_history)}轮]\n")
                    self.last_text = ""
                    
            except json.JSONDecodeError:
                continue
            except Exception as e:
                print(f"消息处理错误: {e}")

def create_assistant(
    db_path: str,
    deepseek_api_key: str,
    tts_url: str,
    asr_url: str,
    debug: bool = True
) -> VoiceAssistant:
    """创建语音助手实例"""
    # 初始化RAG系统
    rag_system = RAGSystem(db_path=db_path, debug=debug)
    
    # 初始化AI客户端
    ai_client = OpenAI(
        api_key=deepseek_api_key,
        base_url="https://api.deepseek.com"
    )
    
    # 创建语音助手
    return VoiceAssistant(
        rag_system=rag_system,
        ai_client=ai_client,
        asr_url=asr_url,
        tts_url=tts_url
    )

import nest_asyncio
nest_asyncio.apply()

def start_assistant(
    deepseek_api_key: str,
    tts_url: str,
    asr_url: str,
    db_path: str,
    debug: bool = True
):
    """启动语音助手"""
    print("开始连接语音服务...")
    print(f"语音识别服务器: {asr_url}")
    print(f"TTS服务器: {tts_url}")
    
    assistant = create_assistant(
        deepseek_api_key=deepseek_api_key,
        tts_url=tts_url,
        asr_url=asr_url,
        db_path=db_path,
        debug=debug
    )
    
    try:
        # 使用asyncio.new_event_loop()创建新的事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
        # 运行助手
        loop.run_until_complete(assistant.start())
    except KeyboardInterrupt:
        print("\n录音已停止")
    except Exception as e:
        print(f"发生错误: {e}")
        print("\n如需重新启动，请再次运行程序。")
    finally:
        if hasattr(assistant, 'stream') and assistant.stream:
            assistant.stream.stop_stream()
            assistant.stream.close()
        if hasattr(assistant, 'audio') and assistant.audio:
            assistant.audio.terminate()
        
        # 清理事件循环
        try:
            loop.stop()
            loop.close()
        except:
            pass

# 使用示例
if __name__ == "__main__":
    config = {
        "deepseek_api_key": "sk-a753a785ce4f4d418de4caf17e82b629",
        "tts_url": "http://copilot.mynatapp.cc",
        "asr_url": "wss://693e14e788532c17.natapp.cc:54321",
        "db_path": "mental_health_resources.db",
        "debug": True
    }
    start_assistant(**config)

开始连接语音服务...
语音识别服务器: wss://693e14e788532c17.natapp.cc:54321
TTS服务器: http://copilot.mynatapp.cc
加载RAG索引...
索引加载完成！
实时识别: 呃
VAD结果: 呃

资源推荐流程开始:
未检测到触发词
AI回复: 您好，请问有什么可以帮助您的？如果您有任何心理方面的困扰或问题，请随时告诉我，我会尽力为您提供帮助。

[对话历史: 1轮]

实时识别: 问是王医生
VAD结果: ，请问是王医生吗

资源推荐流程开始:
未检测到触发词
AI回复: 您好，我是王医生。请问有什么可以帮您的？

[对话历史: 2轮]

实时识别: 怎么解该种
VAD结果: ？我就想问一下啊，就是如果焦虑失眠这种问题应该怎么解决

资源推荐流程开始:
检测到以下触发词:
- 焦虑 (阈值: 0.6)
- 药物 (阈值: 0.7)
- 治疗 (阈值: 0.7)
- 症状 (阈值: 0.6)
使用最高阈值: 0.7

RAG检索过程:
查询文本: ？我就想问一下啊，就是如果焦虑失眠这种问题应该怎么解决 您好，我是王医生。焦虑失眠可以通过规律作息、放松训练和适度运动来缓解。如果症状持续，建议来医院做详细评估，必要时可配合药物治疗。

初始检索结果:
- 社交焦虑自助手册
  初始相似度: 0.0067
  L2距离: 147.2937
- 抗抑郁药物知识普及
  初始相似度: 0.0057
  L2距离: 174.9329
- 抑郁自评量表(SDS)
  初始相似度: 0.0054
  L2距离: 184.5795

重排序过程:

最终返回 0 个结果
AI回复: 您好，我是王医生。焦虑失眠可以通过规律作息、放松训练和适度运动来缓解。如果症状持续，建议来医院做详细评估，必要时可配合药物治疗。

[对话历史: 3轮]

实时识别: 就这样是
VAD结果: ？就这样是吧

资源推荐流程开始:
检测到以下触发词:
- 焦虑 (阈值: 0.6)
- 药物 (阈值: 0.7)
- 治疗 (阈值: 0.7)
使用最高阈值: 0.7

RAG检索过程:
查询文本: ？就这样是吧 是的，我是王医生。焦虑失眠可以通过调整作息、放松训练、适度运动等方法缓解。如果情况严重，建议来医院进行详细评估，可能需要药物治疗或心理治疗